In [1]:
import numpy as np
import pandas as pd

def textToArr(filename):
    arr = []
    with open(filename, "r") as file:
        line = []
        for text in file:
            arr.append(text.strip().split())
    return arr

In [2]:
left = []
right = []

arr = textToArr("1.txt")
for line in arr:
    left.append(int(line[0]))
    right.append(int(line[1]))
    
#sort arrs to match lowest with lowest
left.sort()
right.sort()
sums = 0
for i in range(len(left)):
    sums += abs(left[i] - right[i])

print(sums)

3714264


In [3]:
right_dict = {}

#populate right dict with # of occurences
for val in right:
    if val not in right_dict:
        right_dict[val] = 0
    right_dict[val] += 1

sim_score = 0    
#check all vals in left and multiply by occurences in right
for val in left:
    temp = 0
    if val in right_dict:
        temp = val * right_dict[val]
    sim_score += temp

print(sim_score)

18805872


In [4]:
def getDifferences(level):
    differences = []
    for i in range(len(level) - 1):
        differences.append(level[i + 1] - level[i])
    return differences

def positive(differences):
    return all(val > 0 for val in differences)

def negative(differences):
    return all(val < 0 for val in differences)

def steps(differences):
    return all(1 <= val <= 3 for val in differences)
    
def safeLevel(differences):
    if positive(differences) or negative(differences):
        differences = [abs(val) for val in differences]
        if steps(differences):
            return True
    return False


safe = 0
arr = textToArr("2.txt")
#go level by level
#get differences between elements
#check if all inc or dec
#check if valid step length
for line in arr:
    level = []
    for val in line:
        level.append(int(val))

    if safeLevel(getDifferences(level)):
        safe += 1
        
print(safe)

606


In [5]:
safe = 0
for line in arr:
    dampen = True
    level = []
    for val in line:
        level.append(int(val))

    if safeLevel(getDifferences(level)):
        safe += 1
        dampen = False #unsafe level, try to remove 1 element until safe

    #same logic as pt. 1
    if dampen:
        damp_level = []
        for i in range(len(level)):
            #list comprehension to get new level without each index
            damp_level = [x for j, x in enumerate(level) if j != i]
            if safeLevel(getDifferences(damp_level)):
                safe += 1
                break
            
print(safe)

644


In [6]:
sums = 0
with open("3.txt", "r") as file:
    text = file.read()

#split on mul
#check for ( and )
#check for ,
#try to str --> int
#multiply if all is valid
def doMul(text):
    sums = 0
    splits = text.split("mul")
    for string in splits:
        if string[0] == "(":
            end_idx = string.find(")")
            if end_idx != -1:
                valid = True
                comma_idx = string.find(",")
                a = string[1:comma_idx]
                b = string[comma_idx + 1:end_idx]
                
                try:
                    val_a = int(a)
                    val_b = int(b)
                except:
                    valid = False
                    
                if valid:
                    sums += val_a * val_b
    return sums

print(doMul(text))

173529487


In [7]:
with open("3.txt", "r") as file:
    text = file.read()

#find idx of first substring occurence
#add idx to list, skip to end of substring, loop
def findAll(string, sub):
    i = 0
    idx = []
    while i < len(string):
        i = string.find(sub, i)
        if i == -1: #no more substrings found so just return
            return idx
        idx.append(i)
        i += len(sub)
    return idx
        
#all idx of do() and don't()
do_idx = findAll(text, "do()")
dont_idx = findAll(text, "don't()")

flag_do = True
flag_dont = False
text_modified = ""

#check if current char is the start of a do() or don't()
#update flags for which region we're in
#if in a do() region, then append it to the modified text
for i in range(len(text)):
    if i in do_idx:
        flag_do, flag_dont = True, False
    elif i in dont_idx:
        flag_dont, flag_do = True, False
    
    if flag_do:
        text_modified += text[i]
        
print(doMul(text_modified))

99532691


In [8]:
#2023 day 4
total = 0
text = textToArr("2023.txt")
#go line by line (card by card) in text
#split into left (winning) and right (card nums)
for line in text:
    winning = []
    nums = []
    left = True
    count = 0
    for val in line:
        if val == "|": #reached halfway point, next numbers are right arr (nums)
            left = False 
        try:
            if left:
                winning.append(int(val))
            else:
                nums.append(int(val))
        except:
            continue
    #count how many card nums are in winning
    for num in nums:
        if num in winning:
            count += 1
    if count > 0:
        #get total score
        total += 2 ** (count - 1)
        
print(total)

26443


In [12]:
text = textToArr("2023.txt")
#default dict of all cards default 0 dupes
cards_dict = dict.fromkeys([i for i in range(len(text))], 0)

#go line by line (card by card) in text
#split into left (winning) and right (card nums)
for i in range(len(text)):
    line = text[i]
    winning = []
    nums = []
    left = True
    count = 0
    for val in line:
        if val == "|":
            left = False
        try:
            if left:
                winning.append(int(val))
            else:
                nums.append(int(val))
        except:
            continue
            
    for num in nums:
        if num in winning:
            count += 1

    #if winning card found, then add n (count + dupes) to count # of cards
    if count > 0:
        dupes = cards_dict[i]
        
        for j in range(count):
            try: #make sure card # is valid
                cards_dict[i + j + 1] += 1 + (1 * dupes)
            except:
                break
val = list(cards_dict.values())

#ans is sum of all dupes + each original card
print(sum(val) + len(text))

6284877


In [10]:
#same func for substrings in string, but let index increment by 1 instead
#will find overlapping substrings instead of skipping over the substring

def findAllOverlap(string, sub):
    i = 0
    idx = []
    while i < len(string):
        i = string.find(sub, i)
        if i == -1:
            return idx
        idx.append(i)
        i += 1
    return idx

#count all Xmas in a given line, forwards and backwards
def countXmas(line):
    count = 0
    line = "".join(line)
    count += len(findAllOverlap(line, "XMAS"))
    count += len(findAllOverlap(line[::-1], "XMAS"))
    return count

#function for diag traversal of 2d matrix
#https://www.geeksforgeeks.org/zigzag-or-diagonal-traversal-of-matrix/
def diagonalize(text):
    diag = []
    for line in range(1, (len(text) + len(text[0]))):
        temp = []
        start_col = max(0, line - len(text))
        counts = min(line, (len(text[0]) - start_col), len(text))
        for j in range(0, counts):
            temp.append(text[min(len(text), line) - j - 1][start_col + j]) 
        diag.append(temp)
    return diag

count = 0

#want each char in text to be its own element to form nxn matrix
text = []
with open("4.txt", "r") as file:
    for row in file:
        temp = []
        row = row.strip()
        for char in str(row):
            temp.append(char)
        text.append(temp)

#need to search 4 ways, rows, cols, top left diag, top right diag

#row search
for row in text:
    count += countXmas(row)   

#transpose matrix, then col search
cols = [[text[r][c] for r in range(len(text))] for c in range(len(text[0]))]
for col in cols:
    count += countXmas(col)

#top left diag, then diag search
diag1 = diagonalize(text)
for diag in diag1:
    count += countXmas(diag)

#reverse matrix for top right diag, then diag search
text2 = [row[::-1] for row in text]
diag2 = diagonalize(text2)

for diag in diag2:
    count += countXmas(diag)

print(count)

2390


In [11]:
#same func as above but for "mas" substring
def countMas(line):
    count = 0
    line = "".join(line)
    count += len(findAllOverlap(line, "MAS"))
    count += len(findAllOverlap(line[::-1], "MAS"))
    return count

count = 0

text = []
with open("4.txt", "r") as file:
    for row in file:
        temp = []
        row = row.strip()
        for char in str(row):
            temp.append(char)
        text.append(temp)

# M.S
# .A.
# M.S

#since diags share "A" in the center, search for "A"
#if "A" found, get both diag strings
#if both return a count for "MAS" then we have a match
for r in range(len(text)):
    for c in range(len(text[0])):
        char = text[r][c]
        if char == "A":
            try:
                diag1 = text[r - 1][c - 1] + char + text[r + 1][c + 1]
                diag2 = text[r + 1][c - 1] + char + text[r - 1][c + 1]
                if countMas(diag1) and countMas(diag2):
                    count += 1
            except:
                continue
print(count)

1809


In [101]:
sum = 0
rules = []
first = []
second = []
#get rules in form of [a, b]
with open("5_rules.txt", "r") as file:
    for line in file:
        line = line.strip()
        divide_idx = line.find("|")
        a = int(line[0:divide_idx])
        b = int(line[divide_idx + 1:])
        rules.append([a, b])
        first.append(a)
        second.append(b)

#get pages to be examined
prints = []
with open("5_print.txt", "r") as file:
    for line in file:
        line = line.strip()
        line = line.split(",")
        prints.append([int(val) for val in line])

#check if page is valid
#bubble sort and go through each pair of numbers
#check if the pair is a valid rule in rules
#if valid pair, move on to the next pair
#if not valid pair, then return false
def isValid(page):
    valid = True
    for i in range(len(page)):
        for j in range(i + 1, len(page)):
            rule = [page[i], page[j]]
            if page[j] in second:
                if rule not in rules:
                    valid = False
            elif [page[j], page[i]] in rules:
                valid = False
    return valid

for page in prints:
    print(isValid(page))
    if isValid(page): #sum the middle index of only valid pages
        sum += page[len(page) // 2]

print(sum)

4462


In [104]:
sum = 0
rules = []
first = []
second = []
with open("5_rules.txt", "r") as file:
    for line in file:
        line = line.strip()
        divide_idx = line.find("|")
        a = int(line[0:divide_idx])
        b = int(line[divide_idx + 1:])
        rules.append([a, b])
        first.append(a)
        second.append(b)
        
prints = []
with open("5_print.txt", "r") as file:
    for line in file:
        line = line.strip()
        line = line.split(",")
        prints.append([int(val) for val in line])

for page in prints:
    if not isValid(page): #find invalid pages
        #bubble sort invalid pairs so they are valid
        for i in range(len(page)):
            for j in range(i + 1, len(page)):
                if [page[j], page[i]] in rules:
                    page[j], page[i] = page[i], page[j]
        sum += page[len(page) // 2]

print(sum)

6767
